## Implementation of AlexNet

### Building the model

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
def buildModel(numClasses, seed=2019):
    
    poolZ = 3
    poolS = 2
    
    inputs1 = keras.layers.Input(shape=(227, 227, 3), dtype=tf.float32)
    
    # conv1: 96 kernels 11x11x3, stride=4, padding=valid
    x = keras.layers.Conv2D(96, 11, strides=(4, 4), padding='valid', activation=None,
                            input_shape=(227, 227, 3), name='conv1')(inputs1)
    #TODO: response normalization?!
    # maxpool1: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool1')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu1')(x)
    
    # conv2: 256 kernels 5x5x96, stride=1, padding=same
    x = keras.layers.Conv2D(256, 5, padding='same', activation=None, name='conv2')(x)
    #TODO: response normalization?!
    # maxpool2: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool2')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu2')(x)
    
    # conv3: 384 kernels 3x3x256
    x = keras.layers.Conv2D(384, 3, padding='same', activation=None, name='conv3')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu3')(x)
    
    # conv4: 384 kernels 3x3x384
    x = keras.layers.Conv2D(384, 3, padding='same', activation=None, name='conv4')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu4')(x)    
    
    # conv5: 256 kernels 3x3x384
    x = keras.layers.Conv2D(256, 3, padding='same', activation=None, name='conv5')(x)
    # maxpool5: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool5')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu5')(x)
    
    # automatic flattening...
    x = keras.layers.Flatten('channels_last', name='flatten')(x)
    
    # fc1: 4096 units, dropout 0.5
    x = keras.layers.Dense(4096, activation=None, name='fc1')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='fc1_relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x)
    
    # fc2: 4096 units, dropout 0.5
    x = keras.layers.Dense(4096, activation=None, name='fc2')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='fc2_relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x) 
    
    # fc3: numClasses units
    x = keras.layers.Dense(numClasses, activation='softmax', name='fc3')(x)
    
    outputs = x
    
    return keras.models.Model(inputs = inputs1, outputs = outputs)

### Reading the dataset

In [3]:
import os
import pandas as pd

In [4]:
def readMyDataset(directory):
    # make a list of all image files
    #imgDirectory = os.path.join(directory, 'processed', 'img')
    #listFiles = [os.path.abspath(f) for f in os.scandir(imgDirectory)]
    #print("Found {} image files.".format(len(listFiles)))
    
    descrFile = os.path.join(directory, 'processed', 'fileLabels.txt')
    
    #pathnames = []
    #labelStrings = []
    df = {'pathnames': [], 'labelStrings': []}
    # make a list of all the files we'll process and their labels
    for line in open(descrFile, 'r'):
        #print(line)
        (fileName, label) = line.split(' ')
        # remove quotes and make absolute path
        fileName = os.path.abspath(os.path.join(directory, 'processed', 'img', fileName[1:-1]))
        #print("{} -> {}".format(fileName, label))
        df['pathnames'].append(fileName)
        # remove trailing newline
        df['labelStrings'].append(label[0:-1])
        
    print("Read description of {} files ({} labels).".format(len(df['pathnames']), len(df['labelStrings'])))
    
    #print("ddddd\n{}\ndddddd".format(df['labelStrings'][0]))
    
    df = pd.DataFrame({'pathname': df['pathnames'], 'labelString': df['labelStrings']})
    print("DataFrame: {} rows.".format(len(df.index)))
    
    dummyLabels = pd.get_dummies(df['labelString'], prefix='', prefix_sep='')
    numClasses = len(dummyLabels.columns)
    classes = list(dummyLabels.columns.values)
    print("number of classes: {}".format(numClasses))
    
    dset = tf.data.Dataset.from_tensor_slices((df['pathname'], dummyLabels))
    
    # function to read image data
    def _howToRead(pathname, label):
        imageData = tf.cast(tf.image.resize_images(
            tf.image.decode_png(tf.read_file(pathname), channels=3), [227, 227]), tf.float32)
        return (imageData, label)
    
    def _howToNormalize(img, label):
        return (img / 255.0, label)
    
    def _howToAugRot90(img, label):
        return (tf.image.rot90(img, k=1), label)
    
    def _howToAugRot180(img, label):
        return (tf.image.rot90(img, k=2), label)
    
    def _howToAugRot270(img, label):
        return (tf.image.rot90(img, k=3), label)
    
    def _howToRandomModify(img, label):
        img = tf.image.random_hue(img, 0.1)
        img = tf.image.random_brightness(img, 0.2)
        return (img, label)
    
    def _howToAddRandomNoise(img, label):
        img = img + tf.random.uniform(img.shape, -30, 30)
        return (img, label)
    
    dset = dset.map(_howToRead)
    dsetRandomModified = dset.map(_howToRandomModify)
    #dset = dset.map(_howToNormalize).concatenate(dsetRandomModified.map(_howToNormalize)).concatenate(
    #    dsetRandomModified.map(_howToAugRot90).map(_howToNormalize)).concatenate(
    #    dsetRandomModified.map(_howToAugRot180).map(_howToNormalize)).concatenate(
    #    dsetRandomModified.map(_howToAugRot270).map(_howToNormalize))
    
    dset = dset.concatenate(dsetRandomModified).concatenate(
        dsetRandomModified.map(_howToAugRot90)).concatenate(
        dsetRandomModified.map(_howToAugRot180)).concatenate(
        dsetRandomModified.map(_howToAugRot270))
    dset = dset.concatenate(dset.map(_howToAddRandomNoise))
    dset = dset.map(_howToNormalize)
    
    return (dset, numClasses, classes)
    
    #print("dummies:\n{}".format(dummyLabels))

In [5]:
dataset, numClasses, classes = readMyDataset("/home/ivan/datasets/bilbao2019")

Read description of 3244 files (3244 labels).
DataFrame: 3244 rows.
number of classes: 17
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
print("{}".format(dataset))

<DatasetV1Adapter shapes: ((227, 227, 3), (17,)), types: (tf.float32, tf.uint8)>


In [7]:
print("Classes: {}".format(classes))

Classes: ['ad', 'bench', 'container', 'door', 'extension', 'gate', 'logo', 'manhole', 'socket', 'staircase', 'switch', 'tap', 'textad', 'toilet', 'urinal', 'urn', 'washbasin']


In [8]:
# just to check if everything is ok

iterator = dataset.make_one_shot_iterator()
nextElement = iterator.get_next()

with tf.Session() as sess:
    cnt = 0
    while True:
        try:
            imgbin, label = sess.run(nextElement)
            #print("{}\n".format(imgbin))
            print("Label # {}: {}".format(cnt, label))
            cnt += 1
        except tf.errors.OutOfRangeError:
            break
    

Label # 0: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 2: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 3: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 4: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 5: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 6: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 7: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 8: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 9: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 10: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 11: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 12: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 13: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 14: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 15: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 16: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 17: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 18: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 19: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 20: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
La

Label # 179: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 180: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 181: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 182: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 183: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 184: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 185: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 186: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 187: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 188: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 189: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 190: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 191: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 192: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 193: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 194: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 195: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 196: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 197: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 198: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 199: [0 1 0 

Label # 387: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 388: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 389: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 390: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 391: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 392: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 393: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 394: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 395: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 396: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 397: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 398: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 399: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 400: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 401: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 402: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 403: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 404: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 405: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 406: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 407: [0 0 0 

Label # 577: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 578: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 579: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 580: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 581: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 582: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 583: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 584: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 585: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 586: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 587: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 588: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 589: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 590: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 591: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 592: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 593: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 594: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 595: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 596: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 597: [0 0 0 

Label # 786: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 787: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 788: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 789: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 790: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 791: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 792: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 793: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 794: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 795: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 796: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 797: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 798: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 799: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 800: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 801: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 802: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 803: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 804: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 805: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 806: [0 0 0 

...

Label # 31688: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31689: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31690: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31691: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31692: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31693: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31694: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31695: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31696: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31697: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31698: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31699: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31700: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31701: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31702: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31703: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31704: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31705: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31706: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 31707: [0 0 0 0 0 0 0 0

Label # 31871: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31872: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31873: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31874: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31875: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31876: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31877: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31878: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31879: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31880: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31881: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31882: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31883: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31884: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31885: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31886: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31887: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31888: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31889: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Label # 31890: [0 0 0 0 0 0 1 0

Label # 32046: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32047: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32048: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32049: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32050: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32051: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32052: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32053: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32054: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32055: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32056: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32057: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32058: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32059: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32060: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32061: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32062: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32063: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32064: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32065: [0 0 0 1 0 0 0 0

Label # 32224: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32225: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32226: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32227: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32228: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32229: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32230: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32231: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32232: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32233: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32234: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32235: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32236: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32237: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32238: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32239: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32240: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32241: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32242: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32243: [0 0 0 1 0 0 0 0

Label # 32405: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32406: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32407: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32408: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32409: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32410: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32411: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32412: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32413: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32414: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32415: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32416: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32417: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32418: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Label # 32419: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32420: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32421: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32422: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32423: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 32424: [0 0 0 0 1 0 0 0

### Inputs and outputs

In [9]:
dataset = dataset.shuffle(buffer_size=3000).repeat().batch(32).prefetch(1)

In [10]:
# just to check if everything is ok

#iterator = dataset.make_one_shot_iterator()
#nextElement = iterator.get_next()
#
#with tf.Session() as sess:
#    cnt = 0
#    while True:
#        try:
#            imgbin, label = sess.run(nextElement)
#            #print("{}\n".format(imgbin))
#            print("Label # {}: {}".format(cnt, label))
#            cnt += 1
#        except tf.errors.OutOfRangeError:
#            break

In [11]:
iterator = dataset.make_one_shot_iterator()
inputs, targets = iterator.get_next()
#modelInput = tf.keras.Input(tensor=inputs)
##modelInput = tf.keras.Input(tensor=dataset)

In [12]:
#numClasses = 17
model = buildModel(numClasses)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
targets = tf.cast(targets, tf.float32)

In [14]:
#model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy',
#             metrics=['accuracy'])#, 
             #target_tensors=targets)
model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy',
             metrics=['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
relu1 (Activation)           (None, 27, 27, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 13, 13, 256)       0         
__________

In [17]:
#model.fit(epochs=10, steps_per_epoch=50)
#model.fit(epochs=2, steps_per_epoch=50)
##model.fit(epochs=2, steps_per_epoch=50)
model.fit(inputs, targets, epochs=4, steps_per_epoch=1024)

Instructions for updating:
Use tf.cast instead.
Epoch 1/4
1024/1024 [==============================] - 4033s 4s/step - loss: 0.0250 - acc: 0.9928
Epoch 2/4
1024/1024 [==============================] - 4023s 4s/step - loss: 0.0133 - acc: 0.9967
Epoch 3/4
1024/1024 [==============================] - 4022s 4s/step - loss: 0.0108 - acc: 0.9970
Epoch 4/4
1024/1024 [==============================] - 4026s 4s/step - loss: 0.0078 - acc: 0.9983


### Save model and weights

In [18]:
modelJson = model.to_json()
with open("alexnet_bilbao2019_new1.json", "w") as jsonFile:
    jsonFile.write(modelJson)

In [19]:
#model.save_weights("alexnet_bilbao2019_weights.h5")
model.save_weights("alexnet_bilbao2019_weights_new2.h5")

### Recover weights

In [16]:
model.load_weights("alexnet_bilbao2019_weights_new2.h5")

### Try with some photos

In [21]:
import numpy as np


def tryOnPhoto(model, photoPathname):
    photo = tf.image.decode_jpeg(tf.read_file(photoPathname), channels=3)
    photo = tf.image.resize_images(photo, (227, 227)) / 255.0
    
    modelInput = None
    photoBin = None
    with tf.Session() as sess:
        photoBin = sess.run(photo)
    
    photoBin = np.expand_dims(photoBin, axis=0)
    print(photoBin.shape)
    
    photoBin = tf.constant((photoBin), shape = (1, 227, 227, 3))
    print(photoBin)
    photoBin = tf.data.Dataset.from_tensor_slices(photoBin).batch(1).repeat()
    print(photoBin)
    iterator = photoBin.make_one_shot_iterator()    
    inputs = iterator.get_next()
    #modelInput = keras.layers.Input(tensor=inputs, shape=(1, 227, 227, 3))
        
    print(modelInput)
    return model.predict(inputs, batch_size=1, steps=1, verbose=1)
    #return model.predict(photoBin)

In [28]:
testRes = tryOnPhoto(model, "the-fake-downing-street-10-door.jpg")
#testRes = tryOnPhoto(model, "blue-door-london.jpg")
#testRes = tryOnPhoto(model, "e66858cd-2fcc-41e3-8e4f-5b2b7eb71705.jpeg")
#testRes = tryOnPhoto(model, "contenedores_basura-6-1-1068x712.jpg")
#testRes = tryOnPhoto(model, "1506704365_421985_1506704486_noticia_normal.jpg")
#testRes = tryOnPhoto(model, "staircase_sf1.jpeg")
#testRes = tryOnPhoto(model, "/home/ivan/datasets/bilbao2019/raw/logo/IMG_20190403_094234.jpg")
#testRes = tryOnPhoto(model, "peligro-ratas.jpg")
#testRes = tryOnPhoto(model, "perritoslindos.jpg")
#testRes = tryOnPhoto(model, "contenedores_cerca_auto_basura.jpg_501420591.jpg")
#testRes = tryOnPhoto(model, "cp20f08733531_829923_manual.jpg")
#testRes = tryOnPhoto(model, "5b523b8d018a2.jpeg")
#testRes = tryOnPhoto(model, "Joe-s-Doors-256.jpg")
#testRes = tryOnPhoto(model, "door1.jpg")
#testRes = tryOnPhoto(model, "door2.jpg")
#testRes = tryOnPhoto(model, "door3.jpg")
#testRes = tryOnPhoto(model, "banco2.jpeg")
#testRes = tryOnPhoto(model, "banho1.jpg")
#testRes = tryOnPhoto(model, "banho2.jpg")
#testRes = tryOnPhoto(model, "banho3.jpg")
#inputs, targets = iterator.get_next()
#testRes = model.predict(inputs, batch_size=1, steps=1)

#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/container/IMG_20190408_190520.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/ad/IMG_20190408_184101.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/urn/IMG_20190408_204752.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/toilet/IMG_20190408_114148.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/washbasin/IMG_20190408_113930.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/door/IMG_20190408_205937.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/textad/IMG_20190408_204328.jpg")

(1, 227, 227, 3)
Tensor("Const_3:0", shape=(1, 227, 227, 3), dtype=float32)
<DatasetV1Adapter shapes: (?, 227, 227, 3), types: tf.float32>
None
1/1 [==============================] - 0s 197ms/step


In [29]:
testRes.shape

(1, 17)

In [30]:
print("class: {}, score: {}".format(classes[np.argmax(testRes[0])], np.max(testRes[0])))

class: door, score: 0.999806821346283


In [29]:
print(testRes)

[[1.8752931e-01 1.6534368e-05 1.0426160e-06 7.7682865e-01 1.4696029e-05
  3.7721463e-03 8.2134474e-03 4.2926513e-06 7.7436653e-06 5.4613859e-03
  4.8791226e-05 1.4494960e-06 1.8094145e-02 1.3131172e-06 1.0020865e-07
  1.3969684e-07 4.8575498e-06]]


In [47]:
classes

['ad',
 'bench',
 'container',
 'door',
 'extension',
 'gate',
 'logo',
 'manhole',
 'socket',
 'staircase',
 'switch',
 'tap',
 'textad',
 'toilet',
 'urinal',
 'urn',
 'washbasin']